MIT License

Copyright (c) 2021 Taiki Miyagawa and Akinori F. Ebihara

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Convert and Aggregate Npy Files into a TFRecords File
DATADIR/UCF101TFR[duration]-[240-320 or 256-256]/ will be made.

### Result (trimmed):
```
UCF101TFR-50-240-320:
train01.tfrecords :35996 clips
valid01.tfrecords : 4454 clips
test01.tfrecords  :15807 clips

UCF101TFR-50-256-256:
train01.tfrecords :35996 clips
valid01.tfrecords : 4454 clips
test01.tfrecords  :15807 clips

UCF101TFR-150-240-320:
train01.tfrecords :14467 clips
valid01.tfrecords : 1775 clips
test01.tfrecords  : 6337 clips

UCF101TFR-150-256-256:
train01.tfrecords :14467 clips
valid01.tfrecords : 1775 clips
test01.tfrecords  : 6337 clips
```

In [ ]:
import os, shutil, sys
from glob import glob
import numpy as np
import tensorflow as tf

In [ ]:
# User-defined
DATADIR = "Define this first. E.g., /data/t-miyagawa"

gpu = 7 # which GPU to be used
splitnum = 1 # Official split num. 1, 2, or 3.
duration = 150 # 50 or 150.
target_imshape = 256 # None or 256.
tvt = ["train", "valid", "test"][0] # [0], [1], [2]

In [1]:
# Utility functions
def set_gpu_devices(gpu):
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
    tf.config.experimental.set_visible_devices(physical_devices[gpu], 'GPU')
    tf.config.experimental.set_memory_growth(physical_devices[gpu], True)
    
def np_to_tfr_UCF101(x, y, writer):
    """Save a np.array to a tfrecord file. DO NOT FORGET writer.close().
    Args:
        x: data: np.ndarray, dtype=np.float32. shape=(duration, feat dim)
        y: label: int, dtype=np.int64
        writer: tf.io.TFRecordWriter object. Don't forget writer.close()
    """
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    def _float_feature(value):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

    # Make an Example object that has one record data
    example = tf.train.Example(features=tf.train.Features(feature={
        'video': _bytes_feature(x.tostring()),
        'label': _int64_feature(y)
        }))

    # Serialize the example object and make a TFRecord file
    writer.write(example.SerializeToString())
    
def fix_random_seed(flag_seed, seed=None):
    if flag_seed:
        np.random.seed(seed)
        tf.random.set_seed(seed)
        print("Numpy and TensorFlow's random seeds fixed: seed=" + str(seed))
    
    else:
        print("Random seed not fixed.")

In [11]:
# Get npy paths
ims = "256-256" if target_imshape == 256 else "240-320"
allpaths = sorted(glob("{}/UCF101-{}-{}/{}0{}/*/*.npy".format(DATADIR, duration, ims, tvt, splitnum)))
newdir = "{}/UCF101TFR-{}-{}".format(DATADIR, duration, ims)
record_file = newdir + "/{}0{}.tfrecords".format(tvt, splitnum)

# Get class names in ascending order
classdirs = sorted(glob("{}/UCF101-50-240-320/train01/*".format(DATADIR)))
classnames = [i[i.rfind("train01/") + 8 :] for i in classdirs] # len = num classes

# GPU
set_gpu_devices(gpu)

# Checks
print("The new TFRecords file name: {}".format(record_file))
if len(allpaths) == 0:
    raise ValueError("Any npy file does not exist. See 'allpaths'.")
    
if not os.path.exists(newdir):
    os.makedirs(newdir)
    print("Created a directory: {}".format(newdir))
print("Num of clips: {}".format(len(allpaths)))

The new TFRecords file name: /data/t-miyagawa/UCF101TFR-150-256-256/train01.tfrecords
Num of clips: 14467


In [3]:
# Save a TFRecord file
#############################################
if os.path.exists(record_file):
    raise ValueError("record_file exists. Remove or rename it: {}".format(record_file))
    
# Fix seed and shuffle data
fix_random_seed(True, 7)
shuffler = np.random.permutation(len(allpaths))
print("Shuffle done.")

# Save loop
with tf.io.TFRecordWriter(record_file) as writer:
    for cnt, idx in enumerate(shuffler):
        npy = allpaths[idx]
        label = [i for i, v in enumerate(classnames) if "/"+ v +"/" in npy]
        assert len(label) == 1

        data = np.float32(np.load(npy)) # <== That's it!
        label = np.int64(label[0])      # <== That's it!
        np_to_tfr_UCF101(data, label, writer) 
        
        # Verbose
        if (cnt + 1) % 2000 == 0:
            print("Iter {}/{}: {}".format(cnt + 1, len(allpaths), npy))
            print("Label: {}, Class: {}".format(label, classnames[label]))
print("All saves done.")

Numpy and TensorFlow's random seeds fixed: seed=7
Shuffle done.
Iter 2000/6337: /data/t-miyagawa/UCF101-150-256-256/test01/BrushingTeeth/v_BrushingTeeth_g02_c07_cc03.npy
Label: 19, Class: BrushingTeeth
Iter 4000/6337: /data/t-miyagawa/UCF101-150-256-256/test01/SkateBoarding/v_SkateBoarding_g02_c01_cc01.npy
Label: 79, Class: SkateBoarding
Iter 6000/6337: /data/t-miyagawa/UCF101-150-256-256/test01/MoppingFloor/v_MoppingFloor_g07_c01_cc02.npy
Label: 54, Class: MoppingFloor
All saves done.
